In [2]:
%reset -f
import ipyparallel
import os
rc = ipyparallel.Client(profile="mpi")
view = rc[:]
view.apply(os.chdir, os.getcwd())
view['stop'] = True
print view

view.block=True

<DirectView [0, 1, 2, 3,...]>


In [3]:
%%px --block
from mpi4py import MPI
mpi = MPI.COMM_WORLD
bcast = mpi.bcast
barrier = mpi.barrier
rank = mpi.rank


# Engines Imports
import sys
#!pwd   # Replace sys.path.append with this path
sys.path.append('/home/user/JorgeMijares/air-water-vv/2d/Interactive/3ftFlume_GUI')

import proteus
## Required imports
from proteus.iproteus import * 
from proteus.mprans import SpatialTools as st
from petsc4py import PETSc
from threading import Thread
import nonlinear_waves as plant
import nonlinear_waves_so as plant_so
from proteus import default_n, default_s, default_so, Comm, Context
print "MPI rank: %i/%i" % (mpi.rank,mpi.size)


[stdout:0] MPI rank: 19/20
[stdout:1] MPI rank: 5/20
[stdout:2] MPI rank: 11/20
[stdout:3] MPI rank: 8/20
[stdout:4] MPI rank: 7/20
[stdout:5] MPI rank: 6/20
[stdout:6] MPI rank: 3/20
[stdout:7] MPI rank: 4/20
[stdout:8] MPI rank: 18/20
[stdout:9] MPI rank: 10/20
[stdout:10] MPI rank: 13/20
[stdout:11] MPI rank: 12/20
[stdout:12] MPI rank: 17/20
[stdout:13] MPI rank: 14/20
[stdout:14] MPI rank: 0/20
[stdout:15] MPI rank: 15/20
[stdout:16] MPI rank: 9/20
[stdout:17] MPI rank: 16/20
[stdout:18] MPI rank: 1/20
[stdout:19] MPI rank: 2/20


In [4]:
import numpy as np
def mpi_order(seq):
    """Return elements of a sequence ordered by MPI rank.

    The input sequence is assumed to be ordered by engine ID."""
    ranks = view['rank']
    rank_indices = np.argsort(ranks)
    return [seq[x] for x in rank_indices]


showTriangles = True
showAir = False


def plot_current_results():
    global t_0, plt0
    """Makes a blocking call to retrieve remote data and displays the solution mesh
    as a contour plot.

    Parameters
    ----------
    in_place : bool
        By default it calls clear_output so that new plots replace old ones.  Set
        to False to allow keeping of all previous outputs.
    """
    import numpy as np
    import matplotlib.tri as mtri
    import math
    global nn, x, y, vof, triangles, t, phi, u, v, cfl, p, dt
    view.apply_sync(load_simulation_globals)
    x = np.concatenate(mpi_order(view['x']))
    y = np.concatenate(mpi_order(view['y']))
    u = np.concatenate(mpi_order(view['u']))
    v = np.concatenate(mpi_order(view['v']))
    vof = np.concatenate(mpi_order(view['vof']))
    shifts = np.cumsum([0] + mpi_order(view['nn'])[:-1])
    flat_triangles = np.concatenate([tri + shift for tri, shift in zip(mpi_order(view['triangles']), shifts)])

    t = mpi_order(view['t'])[0]
    phi = np.concatenate(mpi_order(view['phi']))
    cfl = mpi_order(view['cfl'])[0]
    p = np.concatenate(mpi_order(view['p']))
    dt = mpi_order(view['dt'])[0]

    Vmax = np.amax(np.sqrt(u[:] ** 2 + v[:] ** 2))
    print "u_max={0:.3f}, v_max={1:.3f}, Vmax={2:.3f}, cfl={3:.3f}, dt={4:.5f}".format(np.amax(u[:]),
                                                                                       np.amax(v[:]),
                                                                                       Vmax,
                                                                                       np.asarray(cfl).max() * dt,
                                                                                       dt)                                                                                       

def load_simulation_globals():
    """Put some variables we need in engine namespace.

    These can then be retrieved by clients for inspection, visualization, etc.
    """
    global nn, x, y, vof, triangles, t, phi, u, v, cfl, p, dt
    model_vof = ns.modelList[1].levelModelList[-1]
    model_ls = ns.modelList[2].levelModelList[-1]
    # save solution and grid data for plotting purposes
    cfl = ns.modelList[0].levelModelList[-1].q[('cfl', 0)]
    dt = ns.systemStepController.dt_system
    x = ns.modelList[0].levelModelList[-1].mesh.nodeArray[:, 0]
    y = ns.modelList[0].levelModelList[-1].mesh.nodeArray[:, 1]
    triangles = ns.modelList[0].levelModelList[-1].mesh.elementNodesArray
    p = ns.modelList[0].levelModelList[-1].u[0].dof
    u = ns.modelList[0].levelModelList[-1].u[1].dof
    v = ns.modelList[0].levelModelList[-1].u[2].dof
    vof = ns.modelList[1].levelModelList[-1].u[0].dof_last
    phi = ns.modelList[2].levelModelList[-1].u[0].dof_last
    nn = len(x)
    # print "p={0}, u={1}, v={2}, triangles={3}, vof={4}, phi={5}".format(len(p),len(u),len(v),len(triangles),len(vof),len(phi))
    t = ns.systemStepController.t_system
    cfl *= dt


def simulation_alive():
    """Return True if the simulation thread is still running on any engine.
    """
    return any(view.apply_sync(lambda: simulation_thread.is_alive()))

def wait_alive():
    """Return True if the simulation thread is still running on any engine.
    """
    
    import datetime as dt, time
    while not all(view.apply_sync(lambda: simulation_thread.is_alive())):
        print 'Waiting all threads to start...'
        time.sleep(5)
    print "All threads ready"



def monitor_simulation(refresh=5.0):
    global t_0
    """Monitor the simulation progress and call plotting routine.

    Supress KeyboardInterrupt exception if interrupted, ensure that the last 
    figure is always displayed and provide basic timing and simulation status.

    Parameters
    ----------
    refresh : float
      Refresh interval between calls to retrieve and plot data.  The default
      is 5s, adjust depending on the desired refresh rate, but be aware that 
      very short intervals will start having a significant impact.

    """

    import datetime as dt, time
    wait_alive()
    if not simulation_alive():
        plot_current_results()
        print 'Simulation has already finished, no monitoring to do.'
        error = True
        return error
    t_0 = 0.
    t0 = dt.datetime.now()
    try:
        while simulation_alive():
            plot_current_results()
            error = False
            tmon = dt.datetime.now() - t0
            t_sim = view.apply_sync(lambda: ns.systemStepController.t_system)[0]
            print 'Monitored for: %s. at t=%12.5e' % (tmon, t_sim)
            time.sleep(refresh)  # so we don't hammer the server too fast
        print "All threads down"
    except (KeyboardInterrupt):  # , error.TimeoutError):
        msg = 'Monitoring interrupted, simulation is ongoing!'
    else:
        t_sim = view.apply_sync(lambda: ns.systemStepController.t_system)[0]
        if t_sim < txb_tf.value:
            msg = "\x1b[31mStep Failed at t=%12.5e \x1b[0m" % (t_sim)
            error = True
        else:
            msg = 'Simulation completed!'

    tmon = dt.datetime.now() - t0
    print msg
    print 'Monitored for: %s.' % tmon
    return error




In [5]:
def Reload_Context():
        reload(Context)
        reload(plant)
        reload(plant_so)

def Setup_Simulation():
    global ct,ns, simulation_thread

    plant.Update_Model()
    Context.setFromModule(plant,mutable=True)

    plant_so.ct = Context.get()
    ct=plant_so.ct
    so = plant_so
    so.tnList=[0.0,ct.dt_init]+[ct.dt_init+ i*ct.dt_out for i in range(1,ct.nDTout+1)]      
    info = open("TimeList.txt","w")
    for time in so.tnList:
        info.write(str(time)+"\n")
    info.close()
    plant_so.tnList=so.tnList
    pList=[]
    nList=[]
    so.sList=[]
    OptDB = PETSc.Options()
    for (p,n) in so.pnList:
        so.sList.append(default_s)
        pList.append(__import__(p))
        reload(pList[-1])
        nList.append(__import__(n))
        reload(nList[-1])
        pList[-1].name = p
        nList[-1].multilevelLinearSolver = default_n.KSP_petsc4py
        nList[-1].levelLinearSolver = default_n.KSP_petsc4py
        OptDB.setValue(nList[-1].linear_solver_options_prefix+"ksp_type", "preonly")
        OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_type", "lu")
        OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_factor_mat_solver_package","superlu_dist")
    opts.save_dof = True
    opts.dataDir='.'
    opts.probDir='.'
    opts.logLevel=7
    opts.verbose=True
    opts.viewMesh=True


    ## Create numerical solution
    ns = NumericalSolution.NS_base(so, pList, nList, so.sList, opts)


    ## Start Simulation
    simulation_thread = Thread(target = lambda : ns.calculateSolution('run1'))
    simulation_thread.start()


In [6]:


from ipywidgets import *
import ipympl
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
from proteus import Context

reload(Context)
import nonlinear_waves as plant
reload(plant)
import nonlinear_waves_so as plant_so
reload(plant_so)
#


import numpy as np
from IPython.display import clear_output, display,FileLink,FileLinks

from tables import  openFile
from collections import  namedtuple



#import PostProcessing 
#reload(PostProcessing)
import helpers
reload(helpers)
import utilities
reload(utilities)
import PostProcessing
reload(PostProcessing)

from IPython.core.display import HTML
import webbrowser


plt.close("all")

def on_value_change(change):
    clear_output()
    
    view.apply_sync(Reload_Context)
    #Water
    view['plant.opts.water_level'] = waterZ.value
    
    #waves
    view['plant.opts.wave_period'] = sld_WavesPeriod.value
    view['plant.opts.wave_height'] = sld_WavesHeight.value/1000

    # Gauges
    view['plant.opts.gaugeColumn[0]'] = sld_waterColumn1.value
    view['plant.opts.gaugeColumn[1]'] = sld_waterColumn2.value
    
    # Absortion
    view['plant.opts.absorption'] = cb_enAbsorption.value
    
    #Change Context
    plant.opts.water_level=waterZ.value

#     plant.opts.waves=CB_addWaves.value
    plant.opts.wave_period=sld_WavesPeriod.value
    plant.opts.wave_height=sld_WavesHeight.value/1000

    #Gauges
    plant.opts.gaugeColumn[0]=sld_waterColumn1.value
    plant.opts.gaugeColumn[1]=sld_waterColumn2.value
    
    #Absortion
    plant.opts.absorption=cb_enAbsorption.value
    
    
    plant.Update_Model()
    
    Context.setFromModule(plant,mutable=True)
    ct = Context.get()
    plant_so.ct=ct
    
    #Plot new tank
    helpers.f=plt.figure(num=1)
    helpers.plot_domain()
    
    
# Create Tank Dimension Widgets   


#Water
waterZ=FloatSlider(min=0.0, max=plant.opts.tank_dim[1], step=0.05, continuous_update=False,value=plant.opts.water_level,
                            description='Water Level (m):')

sld_WavesHeight=IntSlider(min=0, max=int(1000*plant.tank_dim[1]),
                            step=1, continuous_update=False,value=1000*plant.opts.wave_height,
                            description='Wave Height (m):')  
sld_WavesPeriod=FloatSlider(min=0, max=plant.tank_dim[0]/10,
                            step=0.01, continuous_update=False,value=plant.opts.wave_period,
                            description='Wave Period (s):')  
sld_waterColumn1=FloatSlider(min=0, max=plant.tank_dim[0],
                            step=0.01, continuous_update=False,value=plant.opts.gaugeColumn[0],
                            description='Gauge 1 position (m):') 
sld_waterColumn2=FloatSlider(min=0, max=plant.tank_dim[0],
                            step=0.01, continuous_update=False,value=plant.opts.gaugeColumn[1],
                            description='Gauge 1 position (m):') 
cb_enAbsorption=widgets.Checkbox(value=False, description='Enable Absortion', disabled=False)


waterBox = VBox([waterZ,sld_WavesHeight,sld_WavesPeriod])
sensorBox = VBox([sld_waterColumn1,sld_waterColumn2])
absorptionBox = VBox([cb_enAbsorption])

page1Controls=HBox([waterBox,sensorBox, absorptionBox])

for children_1 in enumerate(page1Controls.children):
    for children_2 in enumerate(children_1[1].children):
        print children_2[1].observe(on_value_change, names='value')


f1 = plt.figure()
plt.plot([])
f1.clear()

# f1_2 = plt.figure(num=2)
# plt.plot([])
# f1_2.clear()

page1=VBox([page1Controls,f1.canvas])
on_value_change(True)

# Link widget values with the current tank dimension values
#mylink1 = jslink((tankLength, 'value'), (caissonWidth, 'max'))
#mylink2 = jslink((tankHeight, 'value'), (caissonHeight, 'max'))
# mylink3 = jslink((tankHeight, 'value'), (waterZ, 'max'))
#mylink3 = jslink((tankHeight, 'value'), (sld_WavesHeight, 'max'))
#mylink4 = jslink((tankHeight, 'value'), (waterX, 'max'))

#============================================================================================================================
#Physics and Numerics
#Load the modules the define the equations to be solved and the numerical methods to use.
#============================================================================================================================


#Prepare Widgets
txb_he=FloatText(value=plant.opts.he, step=1e-15,
                    description='he', disabled=False)
showTriangles=widgets.Checkbox(value=False, description='Show Triangles', disabled=False)
txb_dtInit=FloatText(value=plant.opts.dt_init,step=.001,
                        description='dt initial',disabled=False)
txb_tf=FloatText(value=plant.opts.T,step=1,
                        description='Simulation Time')
showAirWaterSpeed=widgets.Checkbox(value=False, description='Show Air/Water Speed', disabled=False)
btn_parameters = widgets.Button(description="Run Simulation")

#Button simulate code

def on_button_clicked(b):
    clear_output()
    print("Computing Parameters...")
    
    view['plant.opts.dt_init']       = txb_dtInit.value
    view['plant.opts.T']            = txb_tf.value
    view['plant.opts.he']           = txb_he.value

    
    #attrs = vars(ct.domain.MeshOptions)
    #for item in attrs.items():
    #    print "attr: %s \n value:%s" % item 

    pr_list=view.apply_sync(Setup_Simulation)
    print 'Simulation Started'
                 
    ## Monitor Output
    f2.clear()
    plt.plot([])
    f2.canvas.draw()
    error=monitor_simulation(refresh=5.0)  

    
f2=plt.figure()
f2.clear()
plt.plot([])
f2.canvas.draw()
    
btn_parameters.on_click(on_button_clicked)
Sim_Controls=VBox([txb_tf,
                   txb_dtInit,
                   txb_he,
                   showTriangles,
                   showAirWaterSpeed,
                   btn_parameters])
page3=HBox([Sim_Controls,f2.canvas])



#============================================================================================================================
# Create Post-Processing
#============================================================================================================================
f3=plt.figure()
f3.clear()
plt.plot([])
f3.canvas.draw()
 

btn_createImages=Button(description='Create Images')

def on_btn_createImages_clicked(b):
    clear_output()
    print("Creating Images ...")
    f3.clear()
    !rm ./PostProcessing/phi*png
    PostProcessing.plant=plant
    PostProcessing.plt=plt
    PostProcessing.f=plt.figure(num=3)
    PostProcessing.plant_so=plant_so
    name="nonlinear_waves"
    PostProcessing.Create_Images(name)
    clear_output()
    f3.canvas.draw()
    print('Images Created')

btn_createImages.on_click(on_btn_createImages_clicked)
btn_createVideo=Button(description='Create Video',)
txb_rate=IntText(value=10,step=1,
                        description='Frame Rate')
txb_Name=Text(value='DefaultVideoName',
                placeholder='Type something',
                description='Video Name:',
                disabled=False
                )
def on_btn_createVideo_clicked(b):
    clear_output()
    print("Creating Video...")
    rate=txb_rate.value
    name=txb_Name.value
    name="./PostProcessing/"+name
    !rm -f {name}.mp4; LD_LIBRARY_PATH='' avconv -r {rate} -i  ./PostProcessing/phi%4d.png -vcodec libx264 {name}.mp4 -loglevel quiet
    #PostProcessing.Create_Video(name)
    clear_output()
    print("Click link to see video:")
    display(FileLink(name[2:]+".mp4"))
    
btn_createVideo.on_click(on_btn_createVideo_clicked)

page4=HBox([VBox([btn_createImages,btn_createVideo,txb_Name,txb_rate]), f3.canvas])


#============================================================================================================================
# Widget Display
#============================================================================================================================

# Display Widgets in Tabs
lst=['Bathmetry and SDF','Simulation','Post Processing']
children = [page1,page3,page4]
tab = widgets.Tab(children=children)
[tab.set_title(num, name) for num, name in enumerate(lst)]
display(tab)


Computing Parameters...
Simulation Started
All threads ready
